In [89]:
import os
import sys
import pandas as pd

In [90]:
from global_functions import up_load_shp, make_point, add_geo_info_shp,add_geo_info_gdb

In [91]:
#### תאריך
file_date=pd.Timestamp.today().strftime('%y%m%d')

In [92]:
TAZ_V4_date='240404'

In [93]:
software_folder_location=r'W:\Data\Forecast\Tools\forecast_git\create_forecast_basic'

In [94]:
taz=up_load_shp(r'{}\background_files\TAZ_V4_{}.shp'.format(software_folder_location, TAZ_V4_date))

In [95]:
taz = add_geo_info_shp(taz,taz, software_folder_location, 'jeru_metro_jtmt_border_240402', 'jeru_metro')

In [96]:
taz = add_geo_info_shp(taz,taz, software_folder_location, 'in_jeru', 'in_jerusal')
taz.loc[taz['in_jerusal']==0,'in_jerusal']='no'

In [97]:
taz = add_geo_info_shp(taz,taz, software_folder_location, 'main_secto', 'main_secto')

In [98]:
taz = add_geo_info_shp(taz,taz, software_folder_location, 'SCHN_NAME', 'SCHN_NAME')

In [99]:
taz = add_geo_info_shp(taz,taz, software_folder_location, 'subdistrict2008','ENG_NAME_n')
taz['zonetype']=taz['ENG_NAME_n']

In [100]:
taz = add_geo_info_shp(taz,taz, software_folder_location, 'muni_under_JTMT_ITM', 'Muni_Heb')

In [101]:
taz = add_geo_info_shp(taz,taz, software_folder_location, 'muni_under_JTMT_ITM', 'Sug_Muni')

In [102]:
taz = add_geo_info_shp(taz,taz, software_folder_location, 'muni_under_JTMT_ITM', 'CR_PNIM')

In [103]:
list(taz)

['Taz_num',
 'Taz_name',
 'geometry',
 'jeru_metro',
 'in_jerusal',
 'main_secto',
 'SCHN_NAME',
 'ENG_NAME_n',
 'zonetype',
 'Muni_Heb',
 'Sug_Muni',
 'CR_PNIM']

In [104]:
taz = add_geo_info_shp(taz,taz, software_folder_location, 'highBusinessFlag', 'HighBiz')
taz.rename(columns={'HighBiz': 'highBusinessFlag'}, inplace=True)

In [105]:
taz=add_geo_info_shp(taz,taz,software_folder_location,'schDistrict','ID').rename(columns={'ID':'SCHOOLDISTRICT'})

In [106]:
taz=add_geo_info_shp(taz,taz,software_folder_location,'urban','Urban')

In [108]:
taz=add_geo_info_shp(taz,taz,software_folder_location,'EIProp','EIProp')

In [109]:
col=['poly_puma', 'F3', 'F2', 'F1']

for c in col:
    taz=add_geo_info_shp(taz,taz,software_folder_location,'poly_pumas',c)
    

הגעתי לזה זכר לחורבן

col=['agri', 'Indus', 'Com_hotel', 'Business', 'Public']

for c in col:
    print (c)
    taz=add_geo_info_shp(taz,taz,software_folder_location,'emp_category_type',c)
    #taz['{}'.format(c)]=taz['{}'.format(c)]/100

In [116]:
taz = add_geo_info_shp(taz,taz, software_folder_location, 'commuting_230712', 'commuting')
taz['commuting']=taz['commuting']/100

In [117]:
col_parking=['FreeBuffer',
'PaidBuffer',
'Rest_EmpBuffer',
'searchtime',
'walktime',
'cost']

for c in col_parking:
    taz=add_geo_info_gdb(taz,taz,software_folder_location,'parking_abm',c,c)

In [118]:
taz.loc[taz['Taz_num'] < 7001, 'Agg_taz_nu'] = taz['Taz_num'] // 100
taz.loc[taz['Taz_num'] >= 7001, 'Agg_taz_nup'] = taz['Taz_num'] // 10

In [ ]:
taz.to_file(r"{}\background_files\TAZ_V4_{}_with_geo_info.shp".format(software_folder_location, file_date), encoding='utf-8')